Performance metrics are used to evaluate the performance of a machine learning model. These metrics are used to assess how well the model is performing on a given dataset, and they help to identify areas where the model can be improved. Here, it is a classification problem, where the goal is to predict the acceptability of a car based on its attributes. A commonly used scoring metric for classification problems is accuracy, which measures the proportion of correct predictions out of all predictions made. However, accuracy can be misleading if the classes are imbalanced or if the cost of misclassifying one class is much higher than another.
In the context of our problem, a Type 1 error would occur when the model incorrectly predicts that a car is acceptable, good, or very good when it is actually unacceptable. In other words, a Type 1 error is a false positive where the model incorrectly identifies a car as being of higher acceptability than it actually is. A Type 2 error would occur when the model incorrectly predicts that a car is unacceptable when it is actually acceptable, good, or very good. In other words, a Type 2 error is a false negative where the model incorrectly identifies a car as being of lower acceptability than it actually is.

Here as Type 2 error is dominating, we will proceed with "Recall" as the best scoring measure.

Now, we are going to fit a logistic regression, an SVM, and a decision tree model for modeling and use random and grid searches to test a range of parameter values for each model.

# Importing the libraries

In [13]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

np.random.seed(1)

# Load the Data

In [14]:
X_test = pd.read_csv("C:/Users/vvenk/data_test_X.csv")
y_test = pd.read_csv("C:/Users/vvenk/data_test_y.csv")
X_train = pd.read_csv("C:/Users/vvenk/data_train_X.csv")
y_train = pd.read_csv("C:/Users/vvenk/data_train_y.csv")

# Model the data

In [15]:
## First, we will create a dataframe to hold all the results of our models.

performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

# SVM Classification model with Linear Kernel

In [16]:
svm_lin_model = SVC(kernel="linear", probability=True)
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [17]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with linear kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.931624,0.692308,0.692308,0.692308


## SVM Classification model with rbf Kernel

In [18]:
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale', probability=True)
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [19]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with rbf kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.931624,0.692308,0.692308,0.692308
0,svm with rbf kernel,0.977273,0.923077,0.857143,0.888889


# SVM Classification model with Polynomial Kernel

In [20]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10, probability=True)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [21]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with polynomial kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.931624,0.692308,0.692308,0.692308
0,svm with rbf kernel,0.977273,0.923077,0.857143,0.888889
0,svm with polynomial kernel,0.977099,0.923077,0.857143,0.888889


# RandomizedSearchCV SVM

In [22]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10], 
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['linear','poly','rbf']} 
  
rand_search = RandomizedSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
rand_search.fit(X_train, y_train)
print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.698 total time=   0.0s


C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.698 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.698 total time=   0.0s


C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.698 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.701 total time=   0.0s
[CV 1/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.967 total time=   0.0s


...

[CV 5/5] END .....C=10, gamma=0.001, kernel=rbf;, score=0.701 total time=   0.0s
[CV 1/5] END ......C=1, gamma=0.01, kernel=poly;, score=0.698 total time=   0.0s
[CV 2/5] END ......C=1, gamma=0.01, kernel=poly;, score=0.698 total time=   0.0s
[CV 3/5] END ......C=1, gamma=0.01, kernel=poly;, score=0.698 total time=   0.0s
[CV 4/5] END ......C=1, gamma=0.01, kernel=poly;, score=0.698 total time=   0.0s


C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 5/5] END ......C=1, gamma=0.01, kernel=poly;, score=0.701 total time=   0.0s
[CV 1/5] END ......C=10, gamma=1, kernel=linear;, score=0.942 total time=   0.0s
[CV 2/5] END ......C=10, gamma=1, kernel=linear;, score=0.938 total time=   0.0s
[CV 3/5] END ......C=10, gamma=1, kernel=linear;, score=0.942 total time=   0.0s
[CV 4/5] END ......C=10, gamma=1, kernel=linear;, score=0.917 total time=   0.0s


C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 5/5] END ......C=10, gamma=1, kernel=linear;, score=0.921 total time=   0.0s
The best recall score is 0.9851136792291074
... with parameters: {'kernel': 'poly', 'gamma': 1, 'C': 10}


In [23]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random search SVM Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.931624,0.692308,0.692308,0.692308
0,svm with rbf kernel,0.977273,0.923077,0.857143,0.888889
0,svm with polynomial kernel,0.977099,0.923077,0.857143,0.888889
0,Random search SVM Linear,0.984733,0.928571,0.928571,0.928571


# GridSearchCV SVM

In [24]:
score_measure = "recall"
kfolds = 5
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.698 total time=   0.0s


C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.698 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.698 total time=   0.0s


C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.698 total time=   0.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.701 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.698 total time=   0.0s


...

[CV 5/5] END ...C=1000, gamma=0.001, kernel=rbf;, score=0.921 total time=   0.0s
[CV 1/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.897 total time=   0.0s
[CV 2/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.897 total time=   0.0s


C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 3/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.909 total time=   0.0s
[CV 4/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.855 total time=   0.0s
[CV 5/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.888 total time=   0.0s


C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best recall score is 0.9851171084667879
... with parameters: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}


In [25]:
c_matrix = confusion_matrix(y_test, grid.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search SVM Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.931624,0.692308,0.692308,0.692308
0,svm with rbf kernel,0.977273,0.923077,0.857143,0.888889
0,svm with polynomial kernel,0.977099,0.923077,0.857143,0.888889
0,Random search SVM Linear,0.984733,0.928571,0.928571,0.928571
0,Grid search SVM Linear,0.976744,0.923077,0.857143,0.888889


# Decision Tree Classifier

In [26]:
from sklearn.tree import DecisionTreeClassifier

# instantiate the DecisionTreeClassifier model 
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
# fit the model
clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

# Decision tree model using the randomsearch

In [27]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,80),  
    'min_samples_leaf': np.arange(1,40),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is nan
... with parameters: {'min_samples_split': 50, 'min_samples_leaf': 29, 'min_impurity_decrease': 0.004600000000000001, 'max_leaf_nodes': 71, 'max_depth': 34, 'criterion': 'gini'}


C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (

In [28]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.931624,0.692308,0.692308,0.692308
0,svm with rbf kernel,0.977273,0.923077,0.857143,0.888889
0,svm with polynomial kernel,0.977099,0.923077,0.857143,0.888889
0,Random search SVM Linear,0.984733,0.928571,0.928571,0.928571
0,Grid search SVM Linear,0.976744,0.923077,0.857143,0.888889
0,Random search,0.794872,0.263158,0.714286,0.384615


# Decision tree model using the gridsearch

In [29]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(45,55),  
    'min_samples_leaf': np.arange(27,32),
    'min_impurity_decrease': np.arange(0.004, 0.005, 0.0001),
    'max_leaf_nodes': np.arange(70,75), 
    'max_depth': np.arange(32,36), 
    'criterion': ['gini'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits
The best recall score is nan
... with parameters: {'criterion': 'gini', 'max_depth': 32, 'max_leaf_nodes': 70, 'min_impurity_decrease': 0.004, 'min_samples_leaf': 27, 'min_samples_split': 45}


C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the train scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


In [30]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [31]:
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.931624,0.692308,0.692308,0.692308
0,svm with rbf kernel,0.977273,0.923077,0.857143,0.888889
0,svm with polynomial kernel,0.977099,0.923077,0.857143,0.888889
0,Random search SVM Linear,0.984733,0.928571,0.928571,0.928571
0,Grid search SVM Linear,0.976744,0.923077,0.857143,0.888889
0,Random search,0.794872,0.263158,0.714286,0.384615
0,Grid search,0.794872,0.263158,0.714286,0.384615


# Logistic Regression Model

In [33]:
log_reg_model = LogisticRegression()
_ = log_reg_model.fit(X_train, np.ravel(y_train))

In [34]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.931624,0.692308,0.692308,0.692308
0,svm with rbf kernel,0.977273,0.923077,0.857143,0.888889
0,svm with polynomial kernel,0.977099,0.923077,0.857143,0.888889
0,Random search SVM Linear,0.984733,0.928571,0.928571,0.928571
0,Grid search SVM Linear,0.976744,0.923077,0.857143,0.888889
0,Random search,0.794872,0.263158,0.714286,0.384615
0,Grid search,0.794872,0.263158,0.714286,0.384615
0,default logistic,0.905172,0.600000,0.461538,0.521739


# RandomizedSearchCV Logistic Regression

In [35]:
score_measure = "recall"
LR=LogisticRegression()
kfolds = 5
param_grid = {'C': [0.1, 1, 10,0.001], 
              "solver" : [ 'lbfgs', 'liblinear'],
              "penalty" : ['l1','l2','lasso','elastic']} 
  
grid = RandomizedSearchCV(LR, param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END .C=1, penalty=l2, solver=liblinear;, score=0.855 total time=   0.0s
[CV 2/5] END .C=1, penalty=l2, solver=liblinear;, score=0.855 total time=   0.0s
[CV 3/5] END .C=1, penalty=l2, solver=liblinear;, score=0.860 total time=   0.0s
[CV 4/5] END .C=1, penalty=l2, solver=liblinear;, score=0.835 total time=   0.0s
[CV 5/5] END .C=1, penalty=l2, solver=liblinear;, score=0.855 total time=   0.0s
[CV 1/5] END .C=1, penalty=l1, solver=liblinear;, score=0.876 total time=   0.0s
[CV 2/5] END .C=1, penalty=l1, solver=liblinear;

C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 3/5] END .C=0.001, penalty=l2, solver=lbfgs;, score=0.698 total time=   0.0s
[CV 4/5] END .C=0.001, penalty=l2, solver=lbfgs;, score=0.698 total time=   0.0s
[CV 5/5] END .C=0.001, penalty=l2, solver=lbfgs;, score=0.701 total time=   0.0s
[CV 1/5] END ..C=1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ..C=1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ..C=1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ..C=1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ..C=1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ...C=10, penalt

C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

The best recall score is 0.8875038578923906
... with parameters: {'solver': 'liblinear', 'penalty': 'l1', 'C': 10}


In [36]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Randomised", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [37]:
performance

,model,Accuracy,Precision,Recall,F1
0,svm with linear kernel,0.931624,0.692308,0.692308,0.692308
0,svm with rbf kernel,0.977273,0.923077,0.857143,0.888889
0,svm with polynomial kernel,0.977099,0.923077,0.857143,0.888889
0,Random search SVM Linear,0.984733,0.928571,0.928571,0.928571
0,Grid search SVM Linear,0.976744,0.923077,0.857143,0.888889
0,Random search,0.794872,0.263158,0.714286,0.384615
0,Grid search,0.794872,0.263158,0.714286,0.384615
0,default logistic,0.905172,0.600000,0.461538,0.521739
0,Logistic Regression Randomised,0.891892,0.600000,0.428571,0.500000


# GridSearchCV Logistic Regression

In [38]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10], 
              'solver' : [ 'lbfgs', 'liblinear'],
              'penalty' : ['l1','l2','lasso','elastic']} 
  
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.789 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.810 total time=   0.0s
[CV 3/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.814 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.793 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.817 total time=   0.0s
[CV 1/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.806 total time=   0.0s
[CV 2/5] END ...C=0.1, penalty=l2, solver=

C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 1/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5]

C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 4/5] END .....C=1, penalty=l2, solver=lbfgs;, score=0.901 total time=   0.0s
[CV 5/5] END .....C=1, penalty=l2, solver=lbfgs;, score=0.871 total time=   0.0s
[CV 1/5] END .C=1, penalty=l2, solver=liblinear;, score=0.855 total time=   0.0s
[CV 2/5] END .C=1, penalty=l2, solver=liblinear;, score=0.855 total time=   0.0s
[CV 3/5] END .C=1, penalty=l2, solver=liblinear;, score=0.860 total time=   0.0s
[CV 4/5] END .C=1, penalty=l2, solver=liblinear;, score=0.835 total time=   0.0s
[CV 5/5] END .C=1, penalty=l2, solver=liblinear;, score=0.855 total time=   0.0s
[CV 1/5] END ....C=1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ....C=1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ....C=1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ....C=1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ....C=1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=1, penalty=la

...

C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number 

[CV 2/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.913 total time=   0.0s
[CV 3/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.909 total time=   0.0s
[CV 4/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.926 total time=   0.0s
[CV 5/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.905 total time=   0.0s
[CV 1/5] END C=10, penalty=l2, solver=liblinear;, score=0.884 total time=   0.0s
[CV 2/5] END C=10, penalty=l2, solver=liblinear;, score=0.884 total time=   0.0s
[CV 3/5] END C=10, penalty=l2, solver=liblinear;, score=0.864 total time=   0.0s
[CV 4/5] END C=10, penalty=l2, solver=liblinear;, score=0.893 total time=   0.0s
[CV 5/5] END C=10, penalty=l2, solver=liblinear;, score=0.863 total time=   0.0s
[CV 1/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ...C=10, penalt

C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

[CV 4/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .C=10, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=10, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=10, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=10, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=10, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=10, penalty=elastic, solver=liblinear;, score=nan total time=   0.0s
The best re

C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
75 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\vvenk\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Val

In [39]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Grid", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [42]:
performance.sort_values(by =['Recall'])

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression Randomised,0.891892,0.600000,0.428571,0.500000
0,default logistic,0.905172,0.600000,0.461538,0.521739
0,svm with linear kernel,0.931624,0.692308,0.692308,0.692308
0,Random search,0.794872,0.263158,0.714286,0.384615
0,Grid search,0.794872,0.263158,0.714286,0.384615
0,Logistic Regression Grid,0.940678,0.733333,0.785714,0.758621
0,svm with rbf kernel,0.977273,0.923077,0.857143,0.888889
0,svm with polynomial kernel,0.977099,0.923077,0.857143,0.888889
0,Grid search SVM Linear,0.976744,0.923077,0.857143,0.888889
0,Random search SVM Linear,0.984733,0.928571,0.928571,0.928571


Upon comparing "Recall" values of all the models we tried to fit here, we can clearly observe that "Random search SVM Linear" has the highest Recall score (92.8%). We can infer that the model is good at identifying positive samples from the dataset. In particular, recall is a measure of the model's ability to correctly identify all positive samples. A high recall value indicates that the model is capable of capturing the majority of the positive samples in the dataset.

If the recall value for the Random search SVM Linear model is the highest, it means that the model did a good job of correctly identifying cars that are acceptable based on their attributes while minimizing false negatives (i.e., predicting a car as not acceptable when it is actually acceptable).

In future we would need to gather information about a new car's qualities and feed them into the trained model in order to apply this model to it. The trained model would then produce a predicted acceptance rating. It is crucial to remember that the performance of the model itself, together with the perfection and applicability of the data used to train it, will all influence how accurate the forecast will be. Before using the model in real-world scenarios, it is crucial to assess the model's performance on a test set and validate its predictions using fresh data.